In [34]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import mojimoji
import re
import jaconv
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import warnings
from sklearn import preprocessing
import MeCab
def parse_text(text, debug=False):
    text = re.sub(r'［[^］]+］', ' ', text)  
#     text = re.sub(r'（[^）]+）', ' ', text)  
    text = re.sub(r'○', ' ', text)    
    text = re.sub(r'×', '', text)
    text = re.sub(r'※', '', text)    
#     text = re.sub(r'｜', ' ', text)
    text = re.sub(r'[\s、]', '' , text)
#     text = re.sub(r'一', ' ', text)
#     text = re.sub(r'…', ' ', text)  
#     text = re.sub(r'―', ' ', text)
    text = re.sub(r'[0-9]', '0', text)
    return text

def tokenize1(text):
    available_norm = ['接尾', '一般', '形容動詞語幹', 'サ変接続']
    node = mecab.parseToNode(text)
    l = []
    while node:
        l.append(node.surface)
        node = node.next
    return l
traindf=pd.read_csv("data/train.csv")
testdf=pd.read_csv("data/test.csv")
df=pd.concat([traindf,testdf])
df.body=df.body.map(parse_text)
traindf.body=traindf.body.map(parse_text)
testdf.body=testdf.body.map(parse_text)
y = df['author'].values
mecab = MeCab.Tagger('-Owakati')
count = CountVectorizer(analyzer=tokenize1)
bags = count.fit_transform(df.body.values)

features = count.get_feature_names()

bodyvec = pd.DataFrame(bags.toarray(), columns=features)
wordbody=bodyvec.sum().sort_values(ascending=False)
bodyvec1 = bodyvec[list(wordbody[wordbody > 100].index)]
xsum = bodyvec1.sum(axis=1)

xsum = np.array(xsum).reshape(len(xsum), 1)
newdf = pd.concat([df.reset_index(drop=True), pd.DataFrame(bodyvec1)/xsum*100], axis=1)
train=newdf.dropna().drop(["writing_id","body",],axis=1)
test=newdf[newdf.author.isnull()].drop(["writing_id","body"],axis=1)
test=test.drop(["author"],axis=1)
X = train.drop(["author"],axis=1)
y = train.author

mmscaler = preprocessing.MinMaxScaler()

mmscaler.fit(pd.concat([X,test]))
X = mmscaler.transform(X)
test= mmscaler.transform(test)


In [35]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
mlpr=MLPRegressor(**{"hidden_layer_sizes":(128,128,128),"random_state":42})
mlpr.fit(X,y)
pred1 = np.where(mlpr.predict(test)>0.3,1,0)


In [36]:
bodyvec1 = bodyvec[list(wordbody[wordbody > 80].index)]
xsum = bodyvec1.sum(axis=1)

xsum = np.array(xsum).reshape(len(xsum), 1)
newdf = pd.concat([df.reset_index(drop=True), pd.DataFrame(bodyvec1)/xsum*100], axis=1)
train=newdf.dropna().drop(["writing_id","body",],axis=1)
test=newdf[newdf.author.isnull()].drop(["writing_id","body"],axis=1)
test=test.drop(["author"],axis=1)
X = train.drop(["author"],axis=1)
y = train.author

mmscaler = preprocessing.MinMaxScaler()

mmscaler.fit(pd.concat([X,test]))
X = mmscaler.transform(X)
test= mmscaler.transform(test)
mlpr=MLPRegressor(**{"hidden_layer_sizes":(128,128,128),"random_state":42})
mlpr.fit(X,y)
pred2 = np.where(mlpr.predict(test)>0.46,1,0)
# pred=mlpr.predict(test)
# sub = pd.DataFrame(pd.read_csv("data/test.csv")['writing_id'])
# sub["author"] = list(pred)
# sub.to_csv("submission.csv", index = False)

In [37]:
bodyvec1 = bodyvec[list(wordbody[wordbody > 37].index)]
xsum = bodyvec1.sum(axis=1)

xsum = np.array(xsum).reshape(len(xsum), 1)
newdf = pd.concat([df.reset_index(drop=True), pd.DataFrame(bodyvec1)/xsum*100], axis=1)
train=newdf.dropna().drop(["writing_id","body",],axis=1)
test=newdf[newdf.author.isnull()].drop(["writing_id","body"],axis=1)
test=test.drop(["author"],axis=1)
X = train.drop(["author"],axis=1)
y = train.author

mmscaler = preprocessing.MinMaxScaler()

mmscaler.fit(pd.concat([X,test]))
X = mmscaler.transform(X)
test= mmscaler.transform(test)
mlpr=MLPRegressor(**{"hidden_layer_sizes":(128,128,128),"random_state":42})
mlpr.fit(X,y)
pred3 = np.where(mlpr.predict(test)>0.33,1,0)

In [33]:
pred=np.where(pred1+pred2+pred3>1.5,1,0)
sub = pd.DataFrame(pd.read_csv("data/test.csv")['writing_id'])
sub["author"] = list(pred)
sub.to_csv("submission.csv", index = False)

In [16]:
pred=np.where(pred1+pred2>1.1,1,0)
sub = pd.DataFrame(pd.read_csv("data/test.csv")['writing_id'])
sub["author"] = list(pred)
sub.to_csv("submission.csv", index = False)

In [53]:
pred[815]

1

In [11]:
pred2[291]
#291 ✖︎
#682✖︎
#815 ○
#825 ✖︎
#1200 ✖︎

0

In [ ]:
j=0
for i in pred:
    if i==1:
        print(j)
    j+=1

In [32]:
j=0
for i in pred3:
    
    print(i)
    j+=1


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
1
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
1
1
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0


In [23]:
j=0
for i in pred1+pred2+pred3:
    if i>0:
        print(j,i)
    j+=1


19 2
30 2
88 2
96 2
97 2
130 2
134 2
135 2
138 2
141 2
142 2
165 2
167 2
183 2
185 2
186 2
193 2
211 2
218 2
258 2
287 2
291 2
335 2
341 2
343 2
345 2
355 2
380 2
384 2
397 2
412 2
414 2
430 2
435 2
442 2
443 2
448 2
452 2
469 2
485 2
489 2
497 2
506 2
520 2
521 2
526 2
529 2
543 2
544 2
547 2
576 2
581 2
599 2
610 2
621 2
625 2
659 2
682 2
693 2
699 2
702 2
715 2
721 2
746 2
774 2
778 2
784 2
786 2
788 2
794 2
815 2
823 2
825 2
856 2
865 2
869 2
882 2
905 2
921 2
922 2
955 2
965 2
978 2
1003 2
1029 2
1038 2
1078 2
1116 2
1140 2
1146 2
1151 2
1194 2
1200 2
1229 2
1247 2
1266 2
1268 2
1269 2
1281 2
1296 2
1308 2
1322 2
1343 2
1351 2
1354 2
1357 2
1359 2
1361 2
1363 2
1392 2
1400 2
1406 2


In [65]:
j=0
for i in pred1:
    if i>0:
        print(j)
    j+=1


19
30
88
96
97
130
134
135
138
141
142
165
167
183
185
186
193
211
218
258
287
291
335
341
343
345
355
380
384
397
412
414
430
435
442
443
448
452
469
485
489
497
506
520
521
526
529
543
544
547
576
581
599
610
621
625
659
682
693
699
702
715
721
746
774
778
784
786
788
794
815
823
825
856
865
869
882
905
921
922
955
965
978
1003
1029
1038
1078
1116
1140
1146
1151
1194
1200
1229
1247
1266
1268
1269
1281
1296
1308
1322
1343
1351
1354
1357
1359
1361
1363
1392
1400
1406


In [31]:
j=0
pred3 = np.where(mlpr.predict(test)>0.33,1,0)
for i in pred3:
    if i>0.33:
        print(j)
    j+=1

19
30
85
88
96
97
130
134
135
138
141
142
165
167
183
185
186
193
211
218
258
277
287
291
335
341
343
345
355
380
384
397
412
414
430
435
442
443
448
452
469
485
489
497
506
520
521
526
529
543
544
547
576
581
599
610
621
625
659
682
693
699
702
715
721
746
774
778
784
786
788
794
815
823
856
865
869
882
905
921
922
955
965
978
1003
1029
1038
1078
1112
1116
1140
1146
1151
1194
1200
1229
1247
1266
1268
1269
1296
1308
1322
1343
1351
1354
1357
1359
1361
1363
1392
1400
1406
